In [1]:
from notebook_var import column_names, genre_mapping
import pandas as pd
import torch
from torch import nn

In [2]:
class MusicClassifier(nn.Module):
    def __init__(self, input_features, output_features):
        super().__init__()
        self.linear_layer_stack = nn.Sequential(
            nn.Linear(
                in_features=input_features, out_features=256, dtype=torch.float32
            ),
            nn.GELU(),
            nn.Dropout(p=0.2),
            nn.Linear(in_features=256, out_features=128, dtype=torch.float32),
            nn.GELU(),
            nn.Dropout(p=0.2),
            nn.Linear(
                in_features=128, out_features=output_features, dtype=torch.float32
            ),
        )

    def forward(self, x):
        return self.linear_layer_stack(x)

In [3]:
def predict(model: MusicClassifier, df: pd.DataFrame, y_df: pd.DataFrame, genre_mapping: dict[int, str]):
    # TODO: Rewrite
    model.eval()

    class_predictions = []
    raw_results = []
    index_list = []
    total_rows, _ = df.shape

    for i in range(total_rows):
        y_logits = model(
            torch.from_numpy(df.iloc[i].to_numpy().reshape(55, 1).transpose()).type(
                torch.float32
            )
        )

        y_softmax = torch.softmax(y_logits, dim=1)
        y_pred = y_softmax.argmax(dim=1)

        # ! Keep data only if correspond to real class !!!!
        if (y_df.at[i] == y_pred.detach().numpy()[0]):
            index_list.append(i)
            raw_results.append(y_softmax.detach().numpy())
            class_predictions.append(genre_mapping[y_pred.detach().numpy()[0]])
    # !DOnc delete ça ????? 
    # unique_values = set(class_predictions)
    # actual_best = 0
    # for elt in unique_values:
    #     if class_predictions.count(elt) > actual_best:
    #         actual_best = class_predictions.count(elt)
    #         prediction = elt
    return index_list, raw_results

In [4]:
kept_df = pd.DataFrame(columns=column_names)
my_model = MusicClassifier(input_features=55, output_features=10)
my_model.load_state_dict(
    torch.load(
        # f="./resources/actual_model_fast.pth", map_location=torch.device("cpu")
        f="../resources/actual_model_fast.pth", map_location=torch.device("cpu")

    )
)
test_dataframe = pd.read_csv("./resources/test_set.csv").drop(columns=['label'])
y_test = pd.read_csv("./resources/test_set.csv")['label']
index_list, raw_results = predict(my_model, test_dataframe, y_test, genre_mapping)
print(len(raw_results))
print(index_list)
# print("result", result)
# print("raw_results", raw_results)

648
[1, 3, 4, 5, 6, 8, 9, 11, 12, 14, 15, 22, 24, 26, 27, 30, 31, 33, 41, 42, 43, 44, 45, 46, 47, 49, 50, 52, 53, 67, 113, 114, 126, 139, 140, 147, 148, 149, 150, 151, 153, 154, 155, 157, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 170, 171, 173, 175, 176, 177, 178, 179, 180, 181, 182, 183, 186, 187, 188, 190, 194, 197, 218, 219, 239, 243, 245, 246, 267, 274, 275, 276, 277, 279, 280, 281, 282, 283, 284, 285, 286, 287, 288, 289, 290, 291, 292, 293, 294, 295, 299, 305, 306, 307, 308, 309, 310, 311, 312, 314, 315, 316, 319, 322, 323, 324, 325, 326, 327, 328, 329, 330, 331, 332, 333, 334, 335, 336, 339, 340, 341, 342, 344, 349, 350, 351, 353, 354, 355, 356, 357, 361, 362, 363, 364, 367, 368, 369, 370, 372, 373, 374, 375, 376, 377, 378, 379, 380, 381, 382, 383, 384, 385, 386, 387, 388, 389, 390, 391, 392, 393, 394, 395, 396, 398, 424, 426, 441, 442, 457, 513, 519, 520, 560, 567, 577, 585, 590, 592, 596, 607, 625, 628, 664, 667, 669, 674, 723, 724, 725, 726, 727, 730, 731, 732, 767, 77

In [6]:
# ! Garder seulements les lignes de index list dans test-set-dataframe
pd.read_csv("./resources/test_set.csv").iloc[index_list].to_csv("./resources/new_test_set.csv", index=False)